# How to use Google calendar API?

How to import Google calendar API?


In [1]:
var execSync = require('child_process').execSync;
try {
    require.resolve('googleapis');
    require.resolve('google-auth-library');
} catch (e) {
    execSync('npm install googleapis google-auth-library');
}
var google = require('googleapis');
var googleAuth = require('google-auth-library');

var path = require('path');
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var seleniumPath = path.join('..', 'Selenium', 'How to authorize Google apps.ipynb');

// Authorize a client with the loaded credentials, then call the
  // Google Calendar API.
var oauth2Client, authorize, ISODateString;
var getOauthClient = () => {
    return importer.importNotebook(seleniumPath)
        .then(r => authorize = r['authorize'])
        .then(() => authorize(['https://www.googleapis.com/auth/calendar']))
        .then(c => {
            var auth = new googleAuth();
            oauth2Client = new auth.OAuth2(
                c.clientId_, c.clientSecret_, c.redirectUri_);
            oauth2Client.credentials = c.credentials;
            return oauth2Client;
        })
};
(getOauthClient);


[Function: getOauthClient]

How to list events?

In [2]:

/**
 * Lists the next 10 events on the user's primary calendar.
 *
 * @param {google.auth.OAuth2} auth An authorized OAuth2 client.
 */
var resultEvents = [];
var listEvents = (options = {calendarId: 'primary'}) => {
    var auth;
    return getOauthClient()
        .then(r => auth = r)
        .then(() => {
            if(typeof options['calendarId'] == 'undefined') {
                options['calendarId'] = 'primary';
            }
            var calendar = google.calendar('v3');
            var data = Object.assign({}, options, {
                auth: auth,
                //calendarId: 'p4685f6at8ccctenocjlqgnqss@group.calendar.google.com',
                //calendarId: 'rqqhq3me7ggg47chafhmeo6m3s@group.calendar.google.com',
                //q: 'work on SOS',
                maxResults: 100,
                singleEvents: true,
                orderBy: 'startTime'
            });
            return new Promise((resolve, reject) => {
                calendar.events.list(data, (err, response) => {
                    try {
                        var events = processResult(err, response);
                        resolve(events);
                    } catch (e) {
                        reject(e);
                    }
                });
            });
    });
}

(listEvents);


[Function: listEvents]

In [3]:
var path = require('path');
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var interpretPath = path.join('..', 'Core', 'How to interpret all notebooks.ipynb');

var interpret, interpretObject, ISODateString;
var listEventBetweenTwoDates = (min, max) => {
    var startDate = new Date(min);
    var endDate = new Date(max);
    return importer.importNotebook(interpretPath)
    .then((r) => {
        interpretObject = r['interpretObject'];
        interpret = r['interpret'];
        return Promise.all([
            interpret('date to ISO')
        ]);
    })
    .then(r => r.map(r => r[0]))
    .then(r => interpretObject(r))
    .then(r => {
        ISODateString = eval(r[0].code);
        return listEvents({
            timeMin: ISODateString(startDate),
            timeMax: ISODateString(endDate),
        });
    });
};
(listEventBetweenTwoDates);


[Function: listEventBetweenTwoDates]

In [4]:
var processResult = (err, response) => {
    if (err) {
      console.log('The API returned an error: ' + err);
      throw err;
    }
    resultEvents = []; // reset the array
    var events = response.items;
    if (events.length == 0) {
        console.log('No events found.');
    } else {
        for (var i = 0; i < events.length; i++) {
            var event = events[i];
            var start = event.start.dateTime || event.start.date;
            resultEvents[resultEvents.length] = {
                start: new Date(start), event: event};
        }
    }
    return resultEvents;
}


How to test Google calendar API?

In [5]:

$$.async();
getOauthClient()
    .then(r => listEventBetweenTwoDates(new Date('2016/09/22'), new Date('2016/09/23')))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Already imported
[ 'How to import a notebook.ipynb',
  'How to authorize Google apps.ipynb',
  'How to interpret all notebooks.ipynb' ]


[ { start: 2016-09-22T21:00:00.000Z,
    event: 
     { kind: 'calendar#event',
       etag: '"2949158680284000"',
       id: 'orp91edose8auoh3itthj8v4uc',
       status: 'confirmed',
       htmlLink: 'https://www.google.com/calendar/event?eid=b3JwOTFlZG9zZThhdW9oM2l0dGhqOHY0dWMgbWVnYW1pbmRicmlhbkBt',
       created: '2016-09-22T21:21:28.000Z',
       updated: '2016-09-22T21:22:20.142Z',
       summary: 'Established my own legal team',
       description: 'Jim Waring - Jim.Waring@phoenix.gov\nBrad Holm\nPhoenix City Attorney\nCity of Phoenix Law Department\n200 West Washington Street, Suite 1300\nPhoenix, Arizona 85003\n(602) 262-7193 (direct)\n(602) 534-9866 (fax)',
       creator: [Object],
       organizer: [Object],
       start: [Object],
       end: [Object],
       iCalUID: 'orp91edose8auoh3itthj8v4uc@google.com',
       sequence: 0,
       reminders: [Object] } },
  { start: 2016-09-23T00:00:00.000Z,
    event: 
     { kind: 'calendar#event',
       etag: '"2949134836278000"',
